In [1]:
using BenchmarkTools

In [2]:
function Convolution_2d_v1(input, kernel, bias; padding=false)
    input_height, input_width = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        tmp = zeros(input_height+2*kernel_height-2, input_width+2*kernel_width-2)
        for i in 1:input_height
            for j in 1:input_width
                tmp[i+kernel_height-1, j+kernel_width-1] = input[i,j]
            end
        end
        input = tmp
        input_height, input_width = size(input)
    end


    output_height = input_height - kernel_height + 1
    output_width = input_width - kernel_width + 1    
    output = zeros(output_height, output_width)

    for i in 1:output_height
        for j in 1:output_width
            patch = input[i:i+kernel_height-1, j:j+kernel_width-1]
            output[i, j] = sum(patch .* kernel) .+ bias
        end
    end
    return output
end

Convolution_2d_v1 (generic function with 1 method)

In [3]:
function Convolution_2d_v2(input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    output = zeros(Float32, output_rows, output_columns)


    @fastmath for c in 1:output_columns
        @fastmath for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            output[r, c] = sum(patch .* kernel) + bias
        end
    end
    return output
end

Convolution_2d_v2 (generic function with 1 method)

In [4]:
test_input = rand(Float32, 28,28);
test_kernel = rand(Float32, 3, 3);
test_bias = Float32(1);

In [5]:
@benchmark Convolution_2d_v1(test_input, test_kernel, 0; padding=true)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  68.584 μs …  1.871 ms  ┊ GC (min … max): 0.00% … 94.97%
 Time  (median):     69.833 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   72.009 μs ± 26.149 μs  ┊ GC (mean ± σ):  1.37% ±  4.47%

   █▇                                                          
  ▃██▆▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂ ▂
  68.6 μs         Histogram: frequency by time         100 μs <

 Memory estimate: 240.50 KiB, allocs estimate: 1802.

In [6]:
@benchmark Convolution_2d_v2(test_input, test_kernel; bias=test_bias, padding=true)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  39.333 μs … 836.750 μs  ┊ GC (min … max): 0.00% … 90.25%
 Time  (median):     40.250 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   42.043 μs ±  19.098 μs  ┊ GC (mean ± σ):  1.77% ±  3.81%

   ▄██▆▄▃▂▃▃▃▃▂▂ ▁▁     ▁ ▁                                    ▂
  ████████████████████▇██████▇▇▇▇▇▆▆▇▇▇▇▆▅▆▆▇▅▇▆▅▆▄▅▆▅▅▅▄▅▅▆▅▅ █
  39.3 μs       Histogram: log(frequency) by time      54.5 μs <

 Memory estimate: 92.41 KiB, allocs estimate: 904.

In [7]:
Convolution_2d_v1(test_input, test_kernel, 0) == Convolution_2d_v2(test_input, test_kernel)

true